In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

### Import geodata

In [2]:
msoas = gpd.read_file('../90 Data/GLA/MSOA_2011_London_gen_MHW.shp')[['MSOA11CD','MSOA11NM','geometry']] # read shapefile
msoas = msoas.to_crs(epsg = 27700) # set coordinate system to British National Grid
msoas.rename({'MSOA11CD': 'MSOA', 'MSOA11NM': 'MSOA Name'}, axis = 1, inplace = True) # format column names and set index
msoas.set_index('MSOA', inplace = True)
print(len(msoas), 'MSOAs imported')
msoas.head()

983 MSOAs imported


,MSOA Name,geometry
MSOA,,
E02000001,City of London 001,(POLYGON ((531667.6239612588 180534.9919151874...
E02000002,Barking and Dagenham 001,"POLYGON ((548881.5629552712 190845.2649121313,..."
E02000003,Barking and Dagenham 002,"POLYGON ((549102.4379551951 189324.6249117393,..."
E02000004,Barking and Dagenham 003,"POLYGON ((551549.9979543402 187364.6369126641,..."
E02000005,Barking and Dagenham 004,"POLYGON ((549099.6339552177 187656.0759126138,..."


In [3]:
lsoas = gpd.read_file('../90 Data/GLA/LSOA_2011_London_gen_MHW.shp')[['LSOA11CD','MSOA11CD','geometry']] # read shapefile
lsoas = lsoas.to_crs(epsg = 27700) # set coordinate system to British National Grid
lsoas.rename({'LSOA11CD': 'LSOA', 'MSOA11CD': 'MSOA'}, axis = 1, inplace = True)
lsoas.set_index('LSOA', inplace = True)
print(len(lsoas), 'LSOAs imported')
lsoas.head()

4835 LSOAs imported


,MSOA,geometry
LSOA,,
E01000001,E02000001,"POLYGON ((532105.0919603754 182011.229914855, ..."
E01000002,E02000001,"POLYGON ((532746.8129603184 181786.8909149353,..."
E01000003,E02000001,"POLYGON ((532135.1449604381 182198.1189148184,..."
E01000005,E02000001,"POLYGON ((533807.9459600998 180767.7699152774,..."
E01000006,E02000017,"POLYGON ((545122.0489566041 184314.9309138964,..."


In [4]:
londontube = gpd.read_file('../90 Data/misc/londontube.shp')
londontube = londontube.to_crs(epsg = 27700)
londonrail = gpd.read_file('../90 Data/misc/Tracks.shp')
londonrail = londonrail.to_crs(epsg = 27700)

### Import and format population counts data

In [5]:
popcounts = pd.read_csv('../90 Data/UK ONS/msoa_pop_by_age_2017.csv', skiprows = 8, usecols = [i for i in range(1,9)],
                        names = ['MSOA', 'Pop_0_15', 'Pop_16_24', 'Pop_16_17', 'Pop_18_21', 'Pop_25_49', 
                                 'Pop_50_64', 'Pop_65_up'], index_col = 0)
popcounts = popcounts.loc[msoas.index,:] # retain only MSOAs within GLA area
popcounts['Pop_Total'] = np.sum(popcounts[['Pop_0_15','Pop_16_24','Pop_25_49','Pop_50_64','Pop_65_up']], axis = 1) # calculate total figures
popcounts['Pop_WorkingAge'] = np.sum(popcounts[['Pop_16_24','Pop_25_49','Pop_50_64']], axis = 1)
for column in ['Pop_0_15','Pop_16_24','Pop_25_49','Pop_50_64','Pop_65_up','Pop_WorkingAge']: # calculate percent population in each age bin per MSOA
    popcounts['PC_' + column] = popcounts[column] / popcounts['Pop_Total']

print(len(popcounts), 'MSOAs imported with', np.sum(popcounts['Pop_Total']), 'population') # checksum of population numbers
print('Percentages calculated OK:', np.sum(np.sum(popcounts[['PC_Pop_0_15','PC_Pop_16_24','PC_Pop_25_49','PC_Pop_50_64','PC_Pop_65_up']], axis = 1)) == len(popcounts)) # checking that all percentages sum to 1 row-wise
popcounts.describe()

983 MSOAs imported with 8825001 population
Percentages calculated OK: True


,Pop_0_15,Pop_16_24,Pop_16_17,Pop_18_21,Pop_25_49,Pop_50_64,Pop_65_up,Pop_Total,Pop_WorkingAge,PC_Pop_0_15,PC_Pop_16_24,PC_Pop_25_49,PC_Pop_50_64,PC_Pop_65_up,PC_Pop_WorkingAge
count,983.000000,983.000000,983.000000,983.000000,983.000000,983.000000,983.000000,983.000000,983.000000,983.000000,983.000000,983.000000,983.000000,983.000000,983.000000
mean,1844.162767,945.788403,191.807731,381.699898,3721.247202,1409.289929,1057.132248,8977.620549,6076.325534,0.204957,0.104212,0.410553,0.159286,0.120991,0.674052
std,539.457374,407.114632,61.767983,200.812937,1158.473317,300.917603,373.459514,1853.724057,1491.638993,0.038982,0.031895,0.070350,0.029206,0.045259,0.054055
min,599.000000,352.000000,20.000000,129.000000,1479.000000,592.000000,265.000000,5410.000000,3159.000000,0.079696,0.049510,0.249340,0.062505,0.025659,0.547090
25%,1470.500000,714.500000,151.000000,274.000000,2951.000000,1205.000000,779.000000,7742.000000,5045.000000,0.182066,0.086894,0.359728,0.139279,0.086394,0.635066
50%,1758.000000,870.000000,186.000000,345.000000,3536.000000,1393.000000,992.000000,8723.000000,5858.000000,0.203202,0.099358,0.402184,0.158922,0.112222,0.665310
75%,2133.000000,1071.500000,227.000000,430.000000,4298.500000,1587.000000,1280.500000,9972.500000,6807.500000,0.227834,0.112405,0.454610,0.179875,0.147604,0.706842
max,4721.000000,4828.000000,502.000000,2253.000000,13042.000000,2793.000000,2793.000000,23150.000000,19317.000000,0.381189,0.365360,0.640505,0.244327,0.273711,0.848694


### Import and format job counts data

In [6]:
jobcounts = pd.read_csv('../90 Data/UK ONS/msoa_job_by_sector_2017.csv', skiprows = 10, skipfooter = 10,
                        usecols = [1,2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38,40,42],
                        names = ['MSOA', 'Jobs_Agriculture', 'Jobs_Mining', 'Jobs_Manufacturing', 
                                 'Jobs_Energy_Utilities', 'Jobs_Water_Utilities', 'Jobs_Construction', 'Jobs_Wholesale_Retail',
                                 'Jobs_Transport_Storage', 'Jobs_Accom_Food', 'Jobs_Infocomm', 'Jobs_Finance_Insurance',
                                 'Jobs_Real_Estate', 'Jobs_Professional_Scientific_Technical', 'Jobs_Administrative_Support',
                                 'Jobs_Public_Admin_Defence', 'Jobs_Education', 'Jobs_Health_Social_Work', 
                                 'Jobs_Arts_Entertainment', 'Jobs_Other_Services', 'Jobs_Household', 'Jobs_Extraterrestrial_Orgs'],
                        index_col = 0, engine = 'python')
jobcounts['Jobs_Total'] = np.sum(jobcounts, axis = 1) # calculate total jobs

print(len(jobcounts), 'MSOAs imported with', np.sum(jobcounts['Jobs_Total']), 'jobs') # checksum of job numbers
jobcounts.describe()

983 MSOAs imported with 5269290 jobs


,Jobs_Agriculture,Jobs_Mining,Jobs_Manufacturing,Jobs_Energy_Utilities,Jobs_Water_Utilities,Jobs_Construction,Jobs_Wholesale_Retail,Jobs_Transport_Storage,Jobs_Accom_Food,Jobs_Infocomm,...,Jobs_Professional_Scientific_Technical,Jobs_Administrative_Support,Jobs_Public_Admin_Defence,Jobs_Education,Jobs_Health_Social_Work,Jobs_Arts_Entertainment,Jobs_Other_Services,Jobs_Household,Jobs_Extraterrestrial_Orgs,Jobs_Total
count,983.000000,983.000000,983.000000,983.000000,983.000000,983.000000,983.000000,983.000000,983.000000,983.000000,...,983.000000,983.000000,983.000000,983.000000,983.000000,983.000000,983.000000,983.0,983.0,983.000000
mean,1.103764,2.309257,113.189217,11.378433,15.493388,211.505595,636.943032,243.514751,396.887080,426.068159,...,731.429298,591.327569,218.194303,408.280773,573.031536,151.531027,122.293998,0.0,0.0,5360.417091
std,7.219106,22.931937,348.075853,99.367435,66.338527,388.457079,1499.488011,1255.531562,1424.474025,2133.988186,...,4863.086688,2761.406024,1374.997119,714.364432,1276.310961,520.992826,406.229122,0.0,0.0,20188.263515
min,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,10.000000,0.000000,0.000000,5.000000,...,0.000000,10.000000,0.000000,0.000000,10.000000,0.000000,0.000000,0.0,0.0,385.000000
25%,0.000000,0.000000,10.000000,0.000000,0.000000,75.000000,125.000000,20.000000,75.000000,50.000000,...,75.000000,75.000000,0.000000,175.000000,150.000000,20.000000,27.500000,0.0,0.0,1350.000000
50%,0.000000,0.000000,25.000000,0.000000,0.000000,125.000000,300.000000,45.000000,150.000000,100.000000,...,150.000000,150.000000,0.000000,300.000000,250.000000,50.000000,50.000000,0.0,0.0,2300.000000
75%,0.000000,0.000000,75.000000,0.000000,0.000000,225.000000,600.000000,150.000000,300.000000,200.000000,...,300.000000,350.000000,50.000000,450.000000,450.000000,125.000000,100.000000,0.0,0.0,4162.500000
max,125.000000,400.000000,6000.000000,2500.000000,800.000000,8000.000000,31000.000000,28000.000000,25000.000000,45000.000000,...,130000.000000,73000.000000,36000.000000,11000.000000,14000.000000,12000.000000,7000.000000,0.0,0.0,512430.000000


### Import and format household economic indicators

In [7]:
hhincome_mean = pd.read_csv('../90 Data/UK ONS/MSOA-Nett-PostHsgCost-Equivalised-HH-Income-Estimates-2016.csv',
                            skiprows = 5, skipfooter = 1, usecols = [0,6], thousands = ',',
                            names = ['MSOA', 'Est Mean HH Income'], index_col = 0, engine = 'python')
hhincome_mean = hhincome_mean.loc[msoas.index,:] # retain only MSOAs within GLA area
print(len(hhincome_mean), 'MSOAs imported')
hhincome_mean.describe()

983 MSOAs imported


,Est Mean HH Income
count,983.000000
mean,29666.632757
std,7357.419019
min,16600.000000
25%,23700.000000
50%,28300.000000
75%,34500.000000
max,55300.000000


In [8]:
hhpovrate = pd.read_excel('../90 Data/UK ONS/msoa_householdsinpoverty_fye14.xls', sheet_name = 'Households in poverty AHC',
                          skiprows = 5, skipfooter = 1, usecols = [0,6], 
                          names = ['MSOA', 'PC Poor Households']) # Percent of household with income below 60% of median
hhpovrate.set_index('MSOA', inplace = True)
hhpovrate['PC Poor Households'] = hhpovrate['PC Poor Households'] / 100 # convert to decimal
hhpovrate = hhpovrate.loc[msoas.index,:] # retain only MSOAs within GLA area

print(len(hhpovrate), 'MSOAs imported')
hhpovrate.describe()

983 MSOAs imported


,PC Poor Households
count,983.000000
mean,0.260937
std,0.089420
min,0.067000
25%,0.185000
50%,0.257000
75%,0.330500
max,0.502000


### Calculate unemployment rate

In [9]:
unemploymentrate = pd.read_csv('../90 Data/DWP/msoa_alternative_claimant_count_dec2017_feb2019.csv', skiprows = 11, 
                               skipfooter = 11, usecols = [0,1], names = ['MSOA Name','Claimants'], engine = 'python')
unemploymentrate = msoas[['MSOA Name']].merge(unemploymentrate) # join in MSOA IDs
unemploymentrate.set_index(msoas.index, inplace = True)

# calculate MSOA unemployment rate as ratio of alternative claimant count to population aged 16 to 64
unemploymentrate = unemploymentrate.join(popcounts[['Pop_16_24','Pop_25_49','Pop_50_64']])
unemploymentrate['Pop_16_64'] = np.sum(unemploymentrate[['Pop_16_24','Pop_25_49','Pop_50_64']], axis = 1)
unemploymentrate['Unemployment Rate'] = unemploymentrate['Claimants'] / unemploymentrate['Pop_16_64']
unemploymentrate = unemploymentrate[['Claimants','Pop_16_64','Unemployment Rate']]

print(len(unemploymentrate), 'MSOAs imported with', np.sum(unemploymentrate['Claimants']), 'claimants') # checksum of claimant numbers
unemploymentrate.describe()

983 MSOAs imported with 188767 claimants


,Claimants,Pop_16_64,Unemployment Rate
count,983.000000,983.000000,983.000000
mean,192.031536,6076.325534,0.031169
std,99.789687,1491.638993,0.014197
min,22.000000,3159.000000,0.003136
25%,112.000000,5045.000000,0.019915
50%,182.000000,5858.000000,0.030269
75%,256.000000,6807.500000,0.040474
max,619.000000,19317.000000,0.086588


### Calculate MSOA-level Indices of Multiple Deprivation

In [10]:
imd = pd.read_csv('../90 Data/DHCLG/england_indices_multiple_deprivation_lsoa_2015.csv', header = 0, 
                  usecols = [0,4,7,10,13,16,19,22,25,28,31,34,37,40,43,46,49,52,53,54,55,56], index_col = 0)
imd = imd.loc[lsoas.index,:] # subset only LSOAs within GLA boundary
print(len(imd), 'LSOAs imported')
imd.describe()

4835 LSOAs imported


,Index of Multiple Deprivation (IMD) Score,Income Score (rate),Employment Score (rate),"Education, Skills and Training Score",Health Deprivation and Disability Score,Crime Score,Barriers to Housing and Services Score,Living Environment Score,Income Deprivation Affecting Children Index (IDACI) Score (rate),Income Deprivation Affecting Older People (IDAOPI) Score (rate),...,Adult Skills Sub-domain Score,Geographical Barriers Sub-domain Score,Wider Barriers Sub-domain Score,Indoors Sub-domain Score,Outdoors Sub-domain Score,Total population: mid 2012 (excluding prisoners),Dependent Children aged 0-15: mid 2012 (excluding prisoners),Population aged 16-59: mid 2012 (excluding prisoners),Older population aged 60 and over: mid 2012 (excluding prisoners),Working age population 18-59/64: for use with Employment Deprivation Domain (excluding prisoners)
count,4835.000000,4835.000000,4835.000000,4835.000000,4835.000000,4835.000000,4835.000000,4835.000000,4835.000000,4835.000000,...,4835.000000,4835.000000,4835.00000,4835.000000,4835.000000,4835.000000,4835.000000,4835.000000,4835.000000,4835.000000
mean,23.582344,0.165121,0.110709,14.090853,-0.199176,0.546059,29.632529,30.510811,0.229918,0.250612,...,0.260554,-0.625865,3.13945,0.140308,1.027436,1717.675698,344.819235,1109.794209,263.062254,1107.309204
std,12.436196,0.091747,0.057413,10.560608,0.742242,0.568115,9.583645,14.678324,0.128850,0.139193,...,0.106510,0.641087,1.68211,0.561915,0.705989,282.463876,108.364866,249.015295,97.071736,248.804529
min,1.772000,0.007000,0.004000,0.006000,-3.247000,-1.830000,5.811000,4.007000,0.004000,0.007000,...,0.032000,-2.925000,-2.21200,-1.506000,-0.803000,977.000000,42.000000,533.000000,5.000000,562.750000
25%,13.330500,0.090000,0.066000,5.445000,-0.684000,0.172500,22.404500,19.327500,0.121000,0.139000,...,0.178000,-1.078000,1.91000,-0.249000,0.515000,1541.000000,274.000000,940.000000,193.000000,938.250000
50%,22.056000,0.153000,0.102000,12.138000,-0.124000,0.562000,29.193000,29.238000,0.223000,0.231000,...,0.267000,-0.639000,3.22600,0.157000,1.009000,1675.000000,331.000000,1066.000000,252.000000,1062.500000
75%,32.677500,0.229000,0.149000,20.447500,0.345000,0.942000,36.420500,39.563500,0.329000,0.345000,...,0.341500,-0.180000,4.49350,0.518000,1.519000,1848.000000,405.000000,1231.000000,323.000000,1227.250000
max,64.262000,0.461000,0.357000,64.029000,1.846000,2.628000,72.586000,93.352000,0.636000,0.869000,...,0.583000,1.632000,7.72100,2.522000,3.212000,5317.000000,968.000000,4465.000000,704.000000,4608.750000


In [11]:
imd_msoa = imd[['Index of Multiple Deprivation (IMD) Score','Total population: mid 2012 (excluding prisoners)']].join(lsoas[['MSOA']], how = 'left') # join MSOA data
imd_msoa['IMD_Score_x_Pop'] = imd_msoa['Index of Multiple Deprivation (IMD) Score'] * imd_msoa['Total population: mid 2012 (excluding prisoners)'] # calculate weighted sum of IMD score
imd_msoa = imd_msoa.groupby('MSOA').sum() # aggregate to MSOA
imd_msoa['Index of Multiple Deprivation (IMD) Score'] = imd_msoa['IMD_Score_x_Pop'] / imd_msoa['Total population: mid 2012 (excluding prisoners)'] # calculate MSOA weighted mean IMD score
imd_msoa['Index of Multiple Deprivation (IMD) Rank'] = imd_msoa['Index of Multiple Deprivation (IMD) Score'].rank(ascending = False) # rank MSOAs by IMD score, lower rank means more deprived
imd_msoa = imd_msoa[['Index of Multiple Deprivation (IMD) Score','Index of Multiple Deprivation (IMD) Rank','Total population: mid 2012 (excluding prisoners)']]

print(len(imd_msoa), 'MSOAs calculated')
imd_msoa.describe()

983 MSOAs calculated


,Index of Multiple Deprivation (IMD) Score,Index of Multiple Deprivation (IMD) Rank,Total population: mid 2012 (excluding prisoners)
count,983.000000,983.000000,983.000000
mean,23.623171,492.000000,8448.587996
std,10.837233,283.911958,1507.696977
min,3.659492,1.000000,5224.000000
25%,14.478099,246.500000,7437.500000
50%,22.968052,492.000000,8272.000000
75%,31.713005,737.500000,9275.000000
max,54.123421,983.000000,15206.000000


### Write out prepped data

In [12]:
prepped_data = {'msoas': msoas, 
                'lsoas': lsoas, 
                'londontube': londontube,
                'londonrail': londonrail,
                'popcounts': popcounts, 
                'jobcounts': jobcounts, 
                'hhincome_mean': hhincome_mean, 
                'hhpovrate': hhpovrate,
                'unemploymentrate': unemploymentrate, 
                'imd_lsoa': imd,
                'imd_msoa': imd_msoa}

for dataset in prepped_data.keys():
    with open('../90 Data/compiled secondary data/' + dataset, 'wb') as outfile:
        pickle.dump(prepped_data[dataset], outfile)